In [15]:
import os
import matplotlib.pyplot as plt
from nilearn import image as nimg
from nilearn import plotting as nplot
from bids import BIDSLayout
from pathlib import Path
import numpy as np
import pandas as pd
from nilearn import masking
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel

#for inline visualization in jupyter notebook
%matplotlib inline 


In [13]:
def load_data_subject(subject, bids_dir, runs = [1], space = "MNI152NLin2009cAsym"):
    """

    Parameters
    ----------
    subject : str
        Subject identifier e.g. "0102".

    bids_dir : Path
        Path to the root of the bids directory.

    runs : list of int
        List of runs to load.

    space : str
        Name of the space of the data to load.
    

    """
    
    bids_func_dir  = bids_dir / f"sub-{subject}" / "func"
    fprep_func_dir  = bids_dir / "derivatives" / f"sub-{subject}" / "func"
    

    # paths to functional preprocessed data for all runs
    fprep_func_paths = [fprep_func_dir / f"sub-{subject}_task-boldinnerspeech_run-{run}_echo-1_space-{space}_desc-preproc_bold.nii.gz" for run in runs]
    
    # paths to raw functional data for all runs
    raw_func_paths = [bids_func_dir / f"sub-{subject}_task-boldinnerspeech_run-{run}_echo-1_space-{space}_desc-preproc_bold.nii.gz" for run in runs]
    
    events, confounds, masks = [], [], [] 
    
    for f_prep_path, f_raw_path in zip(fprep_func_paths, raw_func_paths):
        # get the corresponding events
        event_path = str(f_raw_path).replace(f'_echo-1_space-{space}_desc-preproc_bold.nii.gz', '_events.tsv')
        event_df = pd.read_csv(event_path, sep='\t')

        # get the data corresponding to the events and only keep the needed columns
        event_df = event_df.loc[:, ["onset", "duration", "trial_type"]]

        # only keep trial types "IMG_POS" and "IMG_NEG"
        event_df = event_df[event_df['trial_type'].isin(['IMG_PO', 'IMG_NO', 'IMG_PS', 'IMG_NS'])]
        events.append(event_df)


        # get the corresponding confounds
        confounds_path = str(f_prep_path).replace(f'_echo-1_space-{space}_desc-preproc_bold.nii.gz', '_desc-confounds_timeseries.tsv')
        confounds_df = pd.read_csv(confounds_path, sep='\t').loc[:, ['trans_x', 'trans_y', 'trans_z', 'rot_x', 'rot_y', 'rot_z']]
        confounds.append(confounds_df)

        # get the corresponding masks
        mask_path = str(f_prep_path).replace(f'_echo-1_space-{space}_desc-preproc_bold.nii.gz', f'_space-{space}_desc-brain_mask.nii.gz')
        mask = nib.load(mask_path)

        masks.append(mask)


    # merge the masks
    mask_img = masking.intersect_masks(masks, threshold=0.8)
    
    # fit the first level model
    first_level_model = FirstLevelModel(t_r=2.0, mask_img=mask_img, verbose=1)

    first_level_model.fit(fprep_func_paths, events, confounds)

    # print the design matrix
    print(first_level_model.design_matrices_[0])

    print("design matrix shape: ", first_level_model.design_matrices_[0].shape)



In [16]:
bids_dir = Path("/work/816119/InSpePosNegData/BIDS_2023E")

# load data for subject 0102
fprep_func_paths, events, confounds, masks = load_data_subject("0122", bids_dir)

Computing run 1 out of 1 runs (go take a coffee, a big one)
/work/LauraBockPaulsen#1941/fMRI_analysis/env/lib/python3.10/site-packages/nilearn/glm/first_level/first_level.py:76: UserWarning: Mean values of 0 observed.The data have probably been centered.Scaling might not work as expected
  warn(


        IMG_NS  IMG_PS   trans_x   trans_y   trans_z     rot_x     rot_y  \
0.0    0.00000     0.0  0.073429  0.039042  0.342614 -0.000362 -0.000157   
2.0    0.00000     0.0  0.080182  0.039424  0.284250  0.000697 -0.000223   
4.0    0.00000     0.0  0.080206 -0.012710  0.245726  0.000854 -0.000191   
6.0    0.00000     0.0  0.069729  0.006396  0.216055  0.000765  0.000028   
8.0    0.00004     0.0  0.060102  0.052535  0.197757  0.001091  0.000287   
...        ...     ...       ...       ...       ...       ...       ...   
970.0  0.00000     0.0 -0.003148 -0.155424  0.307309 -0.015174 -0.000876   
972.0  0.00000     0.0 -0.003190 -0.130248  0.319032 -0.015004 -0.000766   
974.0  0.00000     0.0 -0.003198 -0.103941  0.324269 -0.015556 -0.000591   
976.0  0.00000     0.0 -0.003180 -0.077372  0.351907 -0.016166 -0.000774   
978.0  0.00000     0.0  0.001036 -0.125688  0.295407 -0.015054 -0.000572   

          rot_z   drift_1   drift_2  ...  drift_11  drift_12  drift_13  \
0.0    0.0005


Computation of 1 runs done in 23.199213981628418 seconds.



TypeError: cannot unpack non-iterable NoneType object